In [1]:
import IPython.display as ipd
import sys
sys.path.append('C:\\Users\\quanz\\Documents\\UM\\Projects\\GLX_Project\\DNN\\kelletal2018-master\\kelletal2018-master\\network')
from branched_network_class import branched_network
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()
import scipy.io.wavfile as wav
import matplotlib as plt 
%pylab inline
from pycochleagram import cochleagram as cgram 
from PIL import Image

Instructions for updating:
non-resource variables are not supported in the long term
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


C:\Users\quanz\anaconda3\lib\site-packages\IPython\core\magics\pylab.py:162: UserWarning: pylab import has clobbered these variables: ['plt']
`%matplotlib` prevents importing * from pylab and numpy
  warn("pylab import has clobbered these variables: %s"  % clobbered +
C:\Users\quanz\anaconda3\lib\site-packages\pycochleagram\erbfilter.py:8: RuntimeWarning: pycochleagram using non-interactive Agg matplotlib backend
  from pycochleagram import utils


In [2]:
def generate_cochleagram(wav_f, sr, title):
    # Define parameters
    n, sampling_rate = 50, 16000
    low_lim, hi_lim = 20, 8000
    sample_factor, pad_factor, downsample = 4, 2, 200
    nonlinearity, fft_mode, ret_mode = 'power', 'auto', 'envs'
    strict = True

    # Create cochleagram
    c_gram = cgram.cochleagram(wav_f, sr, n, low_lim, hi_lim, 
                               sample_factor, pad_factor, downsample,
                               nonlinearity, fft_mode, ret_mode, strict)
    
    # Debugging statements
    #print("Cochleagram shape:", c_gram.shape)
    #print("Min value in cochleagram:", np.min(c_gram))
    #print("Max value in cochleagram:", np.max(c_gram))

    # Avoiding log10 issues
    # c_gram[c_gram <= 0] = 1e-10  # Set non-positive values to a small positive number

    # Rescale to [0, 255]
    # c_gram_rescaled = 255 * (1 - ((np.max(c_gram) - c_gram) / np.ptp(c_gram)))

    # Check the shape of c_gram_rescaled
    # print("Shape of c_gram_rescaled:", c_gram_rescaled.shape)

     # rescale to [0,255]
    c_gram_rescaled =  255*(1-((np.max(c_gram)-c_gram)/np.ptp(c_gram)))
    
    # reshape to (256,256)
    c_gram_reshape_1 = np.reshape(c_gram_rescaled, (211,400))
    c_gram_reshape_2 = resample(c_gram_reshape_1,(256,256))
    
    # plot_cochleagram(c_gram_reshape_2, title)

    # prepare to run through network -- i.e., flatten it
    c_gram_flatten = np.reshape(c_gram_reshape_2, (1, 256*256)) 
    
    return c_gram_flatten

In [3]:
def resample(example, new_size):
    im = Image.fromarray(example)
    resized_image = im.resize(new_size, resample=Image.ANTIALIAS)
    return np.array(resized_image)

In [21]:
import numpy as np
from scipy.io import wavfile as wav
import os
import librosa
import pandas as pd

from tqdm import tqdm
from branched_network_class import branched_network
base_path = r'C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\kelletal2018-master\kelletal2018-master'

# Save the current working directory
original_dir = os.getcwd()

# Change to the directory where the weights file is located
os.chdir(r'C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\kelletal2018-master\kelletal2018-master')

# Initialize the network object
net_object = branched_network()
print("Network object created successfully!")


# Load the keys for the model outputs
# word_key = np.load('./demo_stim/logits_to_word_key.npy')  # Speech labels
# music_key = np.load('./demo_stim/logits_to_genre_key.npy')  # Music genre labels

# define the folder where we save all the segments we want to pass into DNN
audio_folder = r'C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\MiND_Stimili\Speech\Segments\Ukranian'

# List of layer names to extract
layer_names = [
    'conv1', 'rnorm1', 'pool1', 'conv2', 'rnorm2', 'pool2',
    'conv3', 'conv4_W', 'conv5_W', 'pool5_W', 'conv4_G', 'conv5_G', 'pool5_G'
]

# Dictionary to store activations for each layer
all_layer_activations = {layer: [] for layer in layer_names}

# make a matrix to save the activation:
    
for filename in tqdm(os.listdir(audio_folder)):
    if filename.endswith('.wav'):
        sr, wav_f = wav.read(os.path.join(audio_folder, filename))
        c_gram = generate_cochleagram(wav_f, sr, filename)

        activations = net_object.get_activations(c_gram)  # Shape: (1, 86, 86, 96)

     # Collect activations for each layer in the specified list
        for layer_name in layer_names:
            layer_activation = activations[layer_name][0]  # Remove batch dimension

            # Flatten the spatial dimensions for the current segment
            flattened_activation = layer_activation.reshape(-1)

            # Append the flattened activation to the list for the current layer
            all_layer_activations[layer_name].append(flattened_activation)

# Set the output directory to the specified path
output_directory = r'C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\MiND_Stimili\Speech\Segments\Ukranian'
os.makedirs(output_directory, exist_ok=True)  # Ensure the directory exists

# Stack activations for each layer across all segments and save as CSV
for layer_name, activations_list in all_layer_activations.items():
    # Stack activations along the segment axis to get an NxS matrix
    stacked_activations = np.stack(activations_list, axis=1)  # Shape: (N, S)

    # Create a DataFrame for easier inspection and saving
    df = pd.DataFrame(stacked_activations, columns=[f'Segment_{i+1}' for i in range(stacked_activations.shape[1])])
    df.index = [f'Unit_{j+1}' for j in range(stacked_activations.shape[0])]  # Label rows by unit

    # Calculate the average activation across segments
    avg_activations = stacked_activations.mean(axis=1)
    df['Average_Activation'] = avg_activations  # Add as a new column

    # Save the matrix with the average column to CSV in the specified output directory
    output_csv = os.path.join(output_directory, f'{layer_name}_activations_with_avg.csv')
    df.to_csv(output_csv)
    print(f"Saved activation matrix with averages for {layer_name} to {output_csv}")

    # Save the average activation as a separate CSV (optional)
    avg_df = pd.DataFrame(avg_activations, columns=['Average_Activation'])
    avg_output_csv = os.path.join(output_directory, f'{layer_name}_average_activation.csv')
    avg_df.to_csv(avg_output_csv, index=False)
    print(f"Saved average activation vector for {layer_name} to {avg_output_csv}")

Network object created successfully!


  0%|                                                                                           | 0/10 [00:00<?, ?it/s]C:\Users\quanz\anaconda3\lib\site-packages\pycochleagram\cochleagram.py:137: RuntimeWarning: divide by zero encountered in log10
  freqs_to_plot = np.log10(freqs)
C:\Users\quanz\AppData\Local\Temp\ipykernel_43192\1703858689.py:3: DeprecationWarning: ANTIALIAS is deprecated and will be removed in Pillow 10 (2023-07-01). Use LANCZOS or Resampling.LANCZOS instead.
  resized_image = im.resize(new_size, resample=Image.ANTIALIAS)
100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:44<00:00,  4.47s/it]


Saved activation matrix with averages for conv1 to C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\MiND_Stimili\Speech\Segments\Ukranian\conv1_activations_with_avg.csv
Saved average activation vector for conv1 to C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\MiND_Stimili\Speech\Segments\Ukranian\conv1_average_activation.csv
Saved activation matrix with averages for rnorm1 to C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\MiND_Stimili\Speech\Segments\Ukranian\rnorm1_activations_with_avg.csv
Saved average activation vector for rnorm1 to C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\MiND_Stimili\Speech\Segments\Ukranian\rnorm1_average_activation.csv
Saved activation matrix with averages for pool1 to C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\MiND_Stimili\Speech\Segments\Ukranian\pool1_activations_with_avg.csv
Saved average activation vector for pool1 to C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\MiND_Stimili\Speech\Segments\Ukranian\pool1_avera

In [9]:
import os

# Check the current working directory
print("Current Working Directory:", os.getcwd())

Current Working Directory: C:\Users\quanz\Documents\UM\Projects\GLX_Project\DNN\kelletal2018-master\kelletal2018-master
